In [ ]:
import numpy as np
import spacy as sp
import pandas as pd
import tweepy as x
import matplotlib.pyplot as plt

In [ ]:
print(np.__version__)

In [ ]:
# # Replace with your actual bearer token
# bearer_token = "AAAAAAAAAAAAAAAAAAAAAB2HzwEAAAAAEOXZJjpw0gbVhuhh34SzqdLAFSg%3DuQfcw16XQ4R5IhggtFuc5n8Cc2EfpDTEOWcZZ1qvHYcWw8SIdI"

# # Initialize the client
# client = x.Client(bearer_token=bearer_token)

# # Define the search query
# # Example: search for tweets containing "Grok", exclude retweets, and only include English tweets.
# query = '(Grok OR #Grok) (opinion OR review OR feedback OR "my experience") -"@" -is:retweet lang:en'

# # Fetch tweets using the client
# # max_results can be between 10 and 100 per request for recent tweets
# response = client.search_recent_tweets(query=query, max_results=50)

# # Check if tweets were returned
# if response.data:
#     for tweet in response.data:
#         print(tweet.text)
# else:
#     print("No tweets found for the query.")

In [ ]:
# annotated_data = { 
#     "sentence": ["@grok @GladiatorMSTR oh my god grok is smart",
#                  "Grok only use information from MSM media propaganda machine. And support that narrative from fake news. It's not neutral. Or free opinion.",
#                  ], 
#     "entities": [{"label": "PERSON", "value": "@grok"},
#                     {"label": "PERSON", "value": "@GladiatorMSTR"},  
#                 {"label": "ARGS", "value": "smart"}] 
    
# }

In [ ]:
import json

# 1. Load spaCy English model
nlp = sp.load("en_core_web_sm")

# 2. Read your file
input_file = "grok-tweet.txt"
with open(input_file, "r", encoding="utf-8") as f:
    lines = f.read().splitlines()

annotated_data = []

# 3. Process each line with spaCy and extract entities
for line in lines:
    if not line.strip():
        continue  # Skip empty lines
    
    doc = nlp(line)
    entities = []
    
    for ent in doc.ents:
        entities.append({
            "start": ent.start_char,
            "end": ent.end_char,
            "label": ent.label_,
            "text": ent.text
        })
    
    annotated_data.append({
        "text": line,
        "entities": entities
    })

# 4. (Optional) Save to a JSON file for later editing
output_file = "annotated_grok.json"
with open(output_file, "w", encoding="utf-8") as out:
    json.dump(annotated_data, out, indent=2, ensure_ascii=False)

print(f"Annotated data saved to {output_file}")


In [ ]:
from textblob import TextBlob

input_file = "grok-tweet.txt"
output_file = "grok-sentiment.json"

with open(input_file, "r", encoding="utf-8") as f:
    lines = f.read().splitlines()

annotated_data = []

for line in lines:
    # Skip empty lines
    if not line.strip():
        continue
    
    # 1. Get sentiment polarity from TextBlob
    blob = TextBlob(line)
    polarity = blob.sentiment.polarity  # -1.0 (negative) -> +1.0 (positive)
    
    # 2. Define simple threshold for sentiment label
    if polarity > 0.1:
        sentiment = "positive"
    elif polarity < -0.1:
        sentiment = "negative"
    else:
        sentiment = "neutral"
    
    annotated_data.append({
        "text": line,
        "polarity": polarity,
        "sentiment": sentiment
    })

# 3. Save to JSON
with open(output_file, "w", encoding="utf-8") as out:
    json.dump(annotated_data, out, indent=2, ensure_ascii=False)

print(f"Saved sentiment results to {output_file}")


In [ ]:
df = pd.read_json("grok-sentiment.json")

In [ ]:
df

In [ ]:
df.to_csv("grok-sentiment.csv", index=False)

In [ ]:
df.to_json("grok-sentiment.json", orient="records", indent=2)

In [ ]:
df = pd.read_json("grok-sentiment.json")

In [ ]:
df

In [ ]:
df = df[~df['text'].str.startswith(' (', na=False)]


In [ ]:
df

In [ ]:
df.to_json("grok-sentiment-cleaned.json", orient="records", indent=4)

In [ ]:
df_cleaned = pd.read_json("grok-sentiment-cleaned.json")

In [ ]:
df_cleaned

In [ ]:
df_cleaned.isna().sum()

In [ ]:
df_cleaned.duplicated().sum()

In [ ]:
df_cleaned.isnull().sum()

In [ ]:
import json

with open("grok-sentiment-cleaned.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)

def get_final_label(row):
    corr = str(row["corrected_sentiment"]).strip().lower()
    if corr in ["positive", "negative", "neutral"]:
        return corr
    else:
        return str(row["sentiment"]).strip().lower()

df["final_sentiment"] = df.apply(get_final_label, axis=1)
df.to_csv("grok-sentiment-final.csv", index=False, encoding="utf-8")

In [ ]:
df

In [ ]:
df.drop(columns=["corrected_sentiment"], inplace=True)

In [ ]:
df

In [ ]:
df.to_csv("grok-sentiment-final-2.csv", index=False, encoding="utf-8")

In [ ]:
# from datasets import Dataset
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# # 1) Load your final CSV
# df = pd.read_csv("grok-sentiment-final.csv")  # or read JSON
# # Must have columns: text, final_sentiment

# # 2) Convert to HF Dataset
# dataset = Dataset.from_pandas(df)

# # 3) Train/Test split
# dataset = dataset.train_test_split(test_size=0.1)

# # 4) Encode labels
# labels = ["negative", "neutral", "positive"]  # or your chosen ordering
# def encode_label(example):
#     example["label"] = labels.index(example["final_sentiment"])
#     return example

# dataset = dataset.map(encode_label)

# # 5) Tokenize text
# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# def tokenize_function(example):
#     return tokenizer(example["text"], truncation=True)

# dataset = dataset.map(tokenize_function, batched=True)

# # 6) Create model & train
# model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(labels))

# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     num_train_epochs=4,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset["train"],
#     eval_dataset=dataset["test"],
# )

# trainer.train()

In [3]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)

# 1) Load your final CSV
df = pd.read_csv("grok-sentiment-final.csv")  
# We expect columns: ["text", "final_sentiment"]

# 2) Convert to a HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# 3) Train/Test split
dataset = dataset.train_test_split(test_size=0.1)

# 4) Encode labels
labels = ["negative", "neutral", "positive"]  # your chosen ordering
def encode_label(example):
    # We'll store the label in "labels" (the standard name HF uses)
    example["labels"] = labels.index(example["final_sentiment"])
    return example

dataset = dataset.map(encode_label)

# 5) Tokenize text
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(batch):
    # Enable padding so each batch has uniform sequence length
    return tokenizer(batch["text"], truncation=True, padding=True)

dataset = dataset.map(tokenize_function, batched=True)

# 6) Remove columns we no longer need, leaving only the model inputs + "labels"
dataset = dataset.remove_columns(["text", "final_sentiment"])

# 7) Create the model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(labels)
)

# 8) Use a data collator for dynamic padding at runtime
data_collator = DataCollatorWithPadding(tokenizer)

# 9) Training arguments (note we use eval_strategy instead of evaluation_strategy)
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",        # rather than evaluation_strategy
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
)

# 10) Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
)

# 11) Train
trainer.train()


Map:   0%|          | 0/401 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Map:   0%|          | 0/401 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,0.676487
2,No log,0.452676
3,No log,0.423105
4,No log,0.464702


TrainOutput(global_step=204, training_loss=0.440793654497932, metrics={'train_runtime': 753.9676, 'train_samples_per_second': 2.127, 'train_steps_per_second': 0.271, 'total_flos': 15355108158696.0, 'train_loss': 0.440793654497932, 'epoch': 4.0})

In [4]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.46470242738723755, 'eval_runtime': 3.9559, 'eval_samples_per_second': 11.376, 'eval_steps_per_second': 1.517, 'epoch': 4.0}


In [5]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [6]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics  # <--- important
)
trainer.train()
eval_results = trainer.evaluate()
print(eval_results)

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.691994,0.844444,0.685185,0.658170,0.665873
2,No log,0.911378,0.844444,0.732733,0.658170,0.686833
3,No log,1.037307,0.800000,0.525132,0.581699,0.551648
4,No log,1.068505,0.822222,0.566066,0.591503,0.578247


{'eval_loss': 1.068505048751831, 'eval_accuracy': 0.8222222222222222, 'eval_precision': 0.5660660660660661, 'eval_recall': 0.5915032679738562, 'eval_f1': 0.5782472613458529, 'eval_runtime': 3.8046, 'eval_samples_per_second': 11.828, 'eval_steps_per_second': 1.577, 'epoch': 4.0}


In [12]:
test_texts = [
    "Grok 3 is insane",
    "what the fuck is this ai bro."
]
# Convert to dataset
test_df = pd.DataFrame({"text": test_texts})
test_dataset = Dataset.from_pandas(test_df)

# Tokenize
test_dataset = test_dataset.map(lambda x: tokenizer(x["text"], truncation=True, padding=True), batched=True)

# Remove extra columns
test_dataset = test_dataset.remove_columns(["text"])

predictions = trainer.predict(test_dataset)
pred_classes = np.argmax(predictions.predictions, axis=1)
for text, pred in zip(test_texts, pred_classes):
    print(text, "->", labels[pred])


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Grok 3 is insane -> positive
what the fuck is this ai bro. -> negative
